In [1]:
import pandas as pd

In [2]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "6Wk1Cny30WOH"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

{'name': '29fc2280ba91',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'gl9GOCGSQDGvLcL0YqBAvw',
 'version': {'distribution': 'opensearch',
  'number': '2.13.0',
  'build_type': 'tar',
  'build_hash': '7ec678d1b7c87d6e779fdef94e33623e1f1e2647',
  'build_date': '2024-03-26T00:02:39.659767978Z',
  'build_snapshot': False,
  'lucene_version': '9.10.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [3]:
import psycopg2
import pandas as pd

# Параметры подключения к базе данных
dbname = 'postgres'
user = 'serikova'
password = '12345'
host = '127.0.0.1'
port = '5432'

# Подключение к базе данных
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Успешное подключение к базе данных PostgreSQL")

    cur = conn.cursor()

    cur.execute("SELECT * from products")
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    df.columns = [desc[0] for desc in cur.description]
    
    cur.close()
    conn.close()

except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении к PostgreSQL:", error)
finally:
    # Закрытие соединения, если оно было установлено
    if conn:
        conn.close()


Успешное подключение к базе данных PostgreSQL


In [4]:
for company, group in df.groupby('product_company'):
    # Здесь можно определить логику выбора значений 'inn' и 'ogrn'
    # Например, берем первые значения в группе
    inn_value = group['inn'].iloc[0]
    ogrn_value = group['ogrn'].iloc[0]
    
    df.loc[df['product_company'] == company, 'inn'] = inn_value
    df.loc[df['product_company'] == company, 'ogrn'] = ogrn_value

In [6]:
import pandas as pd
from sqlalchemy import create_engine

dbname = 'postgres'
user = 'serikova'
password = '12345'
host = '127.0.0.1'
port = '5432'

# Создайте строку подключения к базе данных PostgreSQL
connection_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Установите соединение с базой данных
engine = create_engine(connection_str)
# Загрузите DataFrame в таблицу PostgreSQL
df.to_sql('products', engine, if_exists='replace', index=False)

539

In [10]:
resp = client.search(
    index="products",
    body={
        "query": {
            "bool": {
                "must": {
                    "match": {
                        "product_name": {
                            "query": "клапаан",
                            "fuzziness": "AUTO",
                        }
                    }
                },
            }
        }
    }
)
resp

{'took': 129,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 487, 'relation': 'eq'},
  'max_score': 5.057294,
  'hits': [{'_index': 'products',
    '_id': '2231',
    '_score': 5.057294,
    '_source': {'product_name': 'Клапан',
     'product_desc': 'Цена по запросу',
     'product_price': 'Без минимального объема',
     'product_company': 'ATLAS YEM MAK SAN TIC LTD STI',
     'inn': 7702026003,
     'ogrn': 1720288602538,
     'okpd2': '28.14.11.131'}},
   {'_index': 'products',
    '_id': '2232',
    '_score': 4.6864185,
    '_source': {'product_name': 'Перекидной Клапан',
     'product_desc': 'Цена по запросу',
     'product_price': 'Без минимального объема',
     'product_company': 'ATLAS YEM MAK SAN TIC LTD STI',
     'inn': 7702026003,
     'ogrn': 1720288602538,
     'okpd2': '27.33.11.110'}},
   {'_index': 'products',
    '_id': '2658',
    '_score': 4.6864185,
    '_source': {'product_name': 'Клапан обрат

In [6]:
client.indices.delete(index='products')

{'acknowledged': True}

In [7]:
index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "filter": {
                "russian_stemmer": {
                    "type": "stemmer",
                    "language": "russian"
                }
            },
            "analyzer": {
                "russian_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "russian_synonym_filter",
                        "russian_stemmer",
                        "snowball"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "product_name": {"type": "text", "analyzer": "russian_analyzer"},
            "product_desc": {"type": "text", "analyzer": "russian_analyzer"},
            "product_price": {"type": "keyword"},
            "product_company": {"type": "text", "analyzer": "russian_analyzer"},
            "inn": {"type": "keyword"},
            "ogrn": {"type": "keyword"},
            "okpd2": {"type": "keyword"}
        }
    }
}
# Создание индекса
response = client.indices.create(index="products", body=index_body, ignore=400)  # 400 игнорирует ошибку если индекс уже существует
print(response)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [8]:
import pandas as pd
for i, row in df.iterrows():
    body = {
        "product_name": row["product_name"],
        "product_desc": row["product_desc"],
        "product_price": row["product_price"],
        "product_company": row["product_company"],
        "inn": row["inn"],
        "ogrn": row["ogrn"],
        "okpd2": row["okpd2"]
    }    
    response = client.index(index="products", id=i, body=body)
print(response)

{'_index': 'products', '_id': '27538', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 27538, '_primary_term': 1}
